# BEE 4750 Homework 5: Mixed Integer and Stochastic Programming

**Name**: Jiaming Yuan (jy729), Ari Schor (aes392), Grace Raab (gar238)

**ID**:

> **Due Date**
>
> Thursday, 12/05/24, 9:00pm

## Overview

### Instructions

-   In Problem 1, you will use mixed integer programming to solve a
    waste load allocation problem.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `~/Documents/Programming/Julia/BEE4750/hw/hw5-ari-j`
Precompiling project...
  ✓ Latexify → DataFramesExt
  1 dependency successfully precompiled in 2 seconds. 232 already precompiled.


In [2]:
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

## Problems (Total: 30 Points)

### Problem 1 (30 points)

Three cities are developing a coordinated municipal solid waste (MSW)
disposal plan. Three disposal alternatives are being considered: a
landfill (LF), a materials recycling facility (MRF), and a
waste-to-energy facility (WTE). The capacities of these facilities and
the fees for operation and disposal are provided below.

-   **LF**: Capacity 200 Mg, fixed cost \$2000/day, tipping cost
    \$50/Mg;
-   **MRF**: Capacity 350 Mg, fixed cost \$1500/day, tipping cost
    \$7/Mg, recycling cost \$40/Mg recycled;
-   **WTE**: Capacity 210 Mg, fixed cost \$2500/day, tipping cost
    \$60/Mg;

The MRF recycling rate is 40%, and the ash fraction of non-recycled
waste is 16% and of recycled waste is 14%. Transportation costs are
\$1.5/Mg-km, and the relative distances between the cities and
facilities are provided in the table below.

| **City/Facility** | **Landfill (km)** | **MRF (km)** | **WTE (km)** |
|:-----------------:|:-----------------:|:------------:|:------------:|
|         1         |         5         |      30      |      15      |
|         2         |        15         |      25      |      10      |
|         3         |        13         |      45      |      20      |
|        LF         |        \-         |      32      |      18      |
|        MRF        |        32         |      \-      |      15      |
|        WTE        |        18         |      15      |      \-      |

The fixed costs associated with the disposal options are incurred only
if the particular disposal option is implemented. The three cities
produce 100, 90, and 120 Mg/day of solid waste, respectively, with the
composition provided in the table below.

**Reminder**: Use `round(x; digits=n)` to report values to the
appropriate precision!

**In this problem**:

-   Formulate the waste load allocation problem and implement it in
    `JuMP`.
-   Draw a diagram showing the flows of waste between the cities and the
    facilities. Which facilities (if any) will not be used? Does this
    solution make sense?

In [16]:
waste_model = Model(HiGHS.Optimizer) #initialize model object
#create variables: amount of waste disposed, amount recycled, operational status 
@variable(waste_model, 0 <= W[1:3, 1:3] <= 1.0e10) #W is amount of waste from each city to each disposal plant
@variable(waste_model, 0 <= R[1:3] <= 1.0e10) #R is the amount recycled
@variable(waste_model, 0 <= Y[1:3] <= 1) #Y is operational status of each plant

#create objective function (profit function)
@objective(waste_model, Min, 2000*Y[1]+1500*Y[2]+2500*Y[3]+57.5W[1]+72.5*W[2]+69.5*W[3]+68*W[4]+60.5*W[5]+90.5*W[6]+82.5*W[7]+75*W[8]+90*W[9]+98*R[1]+77*R[2]+82.5*R[3])

#contraints for total area available, avg. rate of pesticide application and maximum total demand for each crop
@constraint(waste_model, cityone, W[1,1]+W[1,2]+W[1,3] == 100)
@constraint(waste_model, citytwo, W[2,1]+W[2,2]+W[2,3] == 90)
@constraint(waste_model, citythree, W[3,1]+W[3,2]+W[3,3] == 120)
@constraint(waste_model, lfmax, W[1,1]+W[2,1]+W[3,1]+R[1]+R[2] <= 200)
@constraint(waste_model, mrfmax, W[1,2]+W[2,2]+W[3,2] <= 350)
@constraint(waste_model, wtemax, W[1,3]+W[2,3]+W[3,3]+R[3] <= 210)
@constraint(waste_model, wteres, R[2]-0.16*(W[1,3]+W[2,3]+W[3,3])-0.14*R[3] == 0)
@constraint(waste_model, mrfres, R[1]+R[3]-0.6*(W[1,2]+W[2,2]+W[3,2]) == 0)
@constraint(waste_model, commit1, Y[1] == 1)
@constraint(waste_model, commit2, !Y[2] => {W[1,2] + W[2,2] + W[3,2] == 0})
@constraint(waste_model, commit3, !Y[3] => {W[1,3]+W[2,3]+W[3,3]+R[3] == 0});

In [22]:
optimize!(waste_model)
@show value.(W)
@show value.(R)
@show value.(Y)
@show value.(cityone)
@show value.(citytwo)
@show value.(citythree)
@show value.(lfmax)
@show value.(mrfmax)
@show value.(wtemax);

Coefficient ranges:
  Matrix [1e-01, 4e+10]
  Cost   [6e+01, 2e+03]
  Bound  [1e+00, 1e+10]
  RHS    [1e+00, 4e+02]
Solving LP without presolve, or with basis, or unconstrained
Model status        : Optimal
Objective value     :  2.5292857151e+04
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00
value.(W) = [100.0 0.0 0.0; 0.0 -0.0 90.0; 79.04761904761904 0.0 40.95238095238096]
value.(R) = [0.0, 20.952380952380956, 0.0]
value.(Y) = [1.0, -0.0, 3.273809523809524e-9]
value.(cityone) = 100.0
value.(citytwo) = 90.0
value.(citythree) = 120.0
value.(lfmax) = 200.0
value.(mrfmax) = 0.0
value.(wtemax) = 130.95238095238096


![](4750hw5_figure.jpeg "Waste Allocation Solution")

### Homework 5 Writeup

The problem objective is to minimize the cost for waste disposal from three cities given three disposal plants: landfill, materials recovery facility, and waste-to-energy. 

...

## References

List any external references consulted, including classmates.

#### References:

To resolve domain error in the model, added boundaries to the variables based on a response from this thread on the Julia discourse page: https://discourse.julialang.org/t/indicator-constraints-in-iqp/107587